In [ ]:
num_workers = 128
budget = 1024
algorithm = 'TwoPointsDE'

In [ ]:
import re
from tempfile import TemporaryDirectory, NamedTemporaryFile

import pandas as pd
import matplotlib.pyplot as plt

from dask.distributed import Client
from dask_jobqueue import SLURMCluster
import nevergrad as ng
import hiplot as hip
from SALib.analyze import delta

from calibrations.synth import obs, model, targets, loss, variables

In [ ]:
cluster = SLURMCluster()
cluster.scale(cores=num_workers)
client = Client(cluster)

In [ ]:
with TemporaryDirectory(dir=systmpfs) as tmpdir:
    sim = model.eval(tmpdir)
    print(f"Before optimization loss is {loss(sim=sim)}")
    for t in targets:
        obs.compare(t, sim, desc=t)
        plt.show()

In [ ]:
kwargs = {name: ng.p.Scalar(init=value.suggested, lower=value.lower, upper=value.upper) 
          for name, value in variables.iterrows()}

optimizer = ng.optimizers.registry[algorithm](parametrization=ng.p.Instrumentation(**kwargs),
                                              budget=budget,
                                              num_workers=num_workers)

logfile = NamedTemporaryFile(dir=systmpfs)
logger = ng.callbacks.ParametersLogger(logfile.name)
optimizer.register_callback("tell",  logger)
optimizer.register_callback("tell", ng.callbacks.ProgressBar())

recommendation = optimizer.minimize(loss_function, executor=client, batch_mode=False)

In [ ]:
samples = pd.DataFrame(logger.load())

samples.dropna(subset=['#loss'], inplace=True)
points = samples[variables.index].to_numpy()
losses = samples['#loss'].to_numpy()

problem = {'num_vars': variables.shape[0],
           'names': variables.index,
           'bounds': list(zip(variables.lower, variables.upper))}

SA = delta.analyze(problem, points, losses)

In [ ]:
variables['best'] = pd.Series(recommendation.kwargs)
variables['err'] = 3 * (variables.synth - variables.best).abs() / (variables.upper - variables.lower)

pd.concat([variables, SA.to_df()], axis=1).sort_values('S1', ascending=False)

In [ ]:
experiment = logger.to_hiplot_experiment()

hidden_columns = ['uid', 
                  'from_uid', 
                  '#parametrization',
                  '#session', 
                  '#lineage',
                  '#meta-sigma']

for name in variables.index:
    hidden_columns.append(name + '#sigma')
    hidden_columns.append(name + '#sigma#sigma')

hidden_columns += [col for col in samples.columns if re.match(r"(#optimizer).*", col)]

table = experiment.display_data(hip.Displays.TABLE)
table.update({'hide': hidden_columns,
              'order_by': [['#num-tell', 'asc']]})

plot = experiment.display_data(hip.Displays.PARALLEL_PLOT)
plot.update({'hide': [*hidden_columns, '#num-tell'],
             'order': ['#generation', *variables.index, '#loss']})

experiment.display()

In [ ]:
with TemporaryDirectory() as tmpdir:
    print(f"After optimization loss is {recommendation.loss}")
    sim = model.eval(tmpdir, **recommendation.kwargs)
    for t in targets:
        obs.compare(t, sim, desc=t)
        plt.show()